In [ ]:
# implement you experimentation code....

## Import the required libraries

In [1]:
import kfp
import kfp.dsl as dsl
from datetime import datetime
import kfp.compiler as compiler

## Load the pipeline components

In [ ]:
component_store = kfp.components.ComponentStore(
  local_search_paths=['components'])

## Create pipeline ops

In [ ]:
add_op = component_store.load_component('my_add')
divide_op = component_store.load_component('my_divide')

## Define pipeline

In [ ]:
def pipeline(
  x_value: int=1,
  y_value: int=1,
  z_value: int=1,
):

  add_step = add_op(x_value=x_value, y_value=y_value)
  add_step.set_display_name('Add x and y')
  add_result = add_step.outputs
  sum_value = add_result['sum']
  is_even = sum_value != 0
  with kfp.dsl.Condition(is_even):
    divide_step = divide_op(x_value=sum_value, y_value=z_value)
    divide_step.set_display_name('Divide sum by z')
    add_step2 = add_op(
      x_value=divide_step.outputs['quotient'],
      y_value=divide_step.outputs['remainder'])
    add_step2.set_display_name('Add quotient and remainder')

## Compile pipeline

In [ ]:
compiler.Compiler().compile(
    pipeline, 'pipeline.tar.gz')

## Run pipeline

In [ ]:
args = {
    'x_value': 1,
    'y_value': 1,
    'z_value': 2,
    
}

client = kfp.Client(namespace='kubeflow')
run_id = 'run-' + datetime.now().strftime('%Y%m%d-%H%M%S')
experiment = client.create_experiment(name='exp-dev')

client.run_pipeline(
    experiment.id,
    job_name=run_id,
    pipeline_package_path=pipeline_package,
    params=args
)